In [ ]:
## https://developers.arcgis.com/python/latest/api-reference/arcgis.raster.utils.html#arcgis.raster.utils.generate_direct_access_url


In [7]:
import arcgis
from arcgis.raster import *
from arcgis.raster.analytics import *
from arcgis.gis import GIS

arcgis.env.verbose=True

In [8]:
# Prompt user to provide username and password
import getpass
username = getpass.getpass(prompt='Username: ')
password = getpass.getpass(prompt='Password: ')

In [9]:
# Connect to ArcGIS Online
gis = GIS("https://esri.maps.arcgis.com/", username, password, verify_cert=False)
gis

Setting `verify_cert` to False is a security risk, use at your own risk.


GIS @ https://esri.maps.arcgis.com version:2024.3

In [17]:
import os
upload_crf = r'E:\test\ssp370\arag_10m_1985.crf'
home_path = os.path.dirname(upload_crf)
print(home_path)
folder_name = upload_crf.split('\\')[-2].replace('.crf','')
crf_name = upload_crf.split('\\')[-1].replace('.crf','')
print(folder_name, crf_name)

E:\test\ssp370
ssp370 arag_10m_1985


In [11]:
# Request direct access URL from ArcGIS Online org with the current log in ArcGIS Online user

from arcgis.raster.utils import *

access_url = arcgis.raster.utils.generate_direct_access_url(expiration=1440, gis=gis)
access_url

'https://stg-arcgisazureimagery.az.arcgis.com/p3eplmys2rvchkjx-8223a93974b7aa6265befa1ca28427d1?sv=2018-03-28&sr=c&sig=%2B7ujUqVLUJ0Mk%2BMlKjmOAM9a4rjYSJZzk2IcXl%2Bmb0I%3D&st=2024-11-19T21%3A33%3A51Z&se=2024-11-20T21%3A38%3A51Z&sp=racwdl'

In [13]:
import arcpy
# Create cloud storage connection file with the information from the direct access URL

folder_name = home_path
target_connection_file = "agol_azure.acs"

account_name = "<account name>"
container_name = "<org id>-<user id>"

config_options = "AZURE_STORAGE_SAS_TOKEN <sas token>;CPL_AZURE_ENDPOINT <azure endpoint>"

source_connection = arcpy.management.CreateCloudStorageConnectionFile(
    out_folder_path=folder_name, 
    out_name=target_connection_file,
    service_provider="AZURE", 
    bucket_name=container_name, 
    access_key_id=account_name,
    config_options=config_options)

In [15]:
import os

folder_name = upload_crf
# src_connection_file = "userstore_azure.acs"
target_connection_file = "agol_azure.acs"
target_path = "/_images/" + folder_name

source = upload_crf
target = os.path.join(target_connection_file, folder_name)

arcpy.management.TransferFiles(source, target)

<Result '\\\\'>

In [22]:
print(target)
print(source)

E:\test\ssp370\arag_10m_1985.crf
E:\test\ssp370\arag_10m_1985.crf


In [20]:
# The following code snippet will convert the uploaded images to a single Tiled Imagery Layer
# by mosaicking multiple images

data_url = access_url.split('?')[0] + "\\" + crf_name
print(data_url)

layer_name = crf_name

ImageryLayer = copy_raster(input_raster=data_url,
                           output_name=layer_name,
                           raster_type_name="Raster Dataset",
                           tiles_only=True,
                           gis=gis)

https://stg-arcgisazureimagery.az.arcgis.com/p3eplmys2rvchkjx-8223a93974b7aa6265befa1ca28427d1\arag_10m_1985


RuntimeError: An Image Service by this name already exists: arag_10m_1985